In [1]:
import cv2
import numpy as np

In [2]:
!pip install opencv-contrib-python

In [3]:
cap = cv2.VideoCapture('testVideo.mp4')
sub = cv2.createBackgroundSubtractorMOG2()
v_detection = []
cmp = 0

def centre(p_x, p_y, width, height):
    x = width/2
    y = height/2
    n_x = int(p_x+x)
    n_y = int(p_y+y)
    return n_x,n_y

In [4]:
while True:
    ret, frame = cap.read()
    
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    d_noise = cv2.medianBlur(grey,7)
    s_frame = sub.apply(d_noise)
    
    dilatation = cv2.dilate(s_frame,np.ones((3,3)))
    elementStruc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    close = cv2.morphologyEx(dilatation,cv2.MORPH_CLOSE,elementStruc)
    close = cv2.morphologyEx(close,cv2.MORPH_CLOSE,elementStruc)
    
    contours,h = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame, (0,300),(639,300),(255,0,0))
    
    for (i,c) in enumerate(contours):
        (p_x, p_y, width, height) = cv2.boundingRect(c)
        if (width < 80) and (height < 80):
            continue
        
        cv2.rectangle(frame, (p_x,p_y), (p_x+width,p_y+height), (0,255,0))
        
        c = centre(p_x,p_y,width,height)
        v_detection.append(c)
        
        for (x,y) in v_detection:
            if y < 305 and y > 295:
                cv2.line(frame, (0,300),(639,300),(0,0,255))
                cmp +=1
                
            v_detection.remove((x,y))
        
    cv2.putText(frame,"num: "+str(cmp),(320,254),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
    
    cv2.imshow(' ',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break

cv2.destroyAllWindows()
cap.release()